In [1]:
from builtins import str

from netrc import netrc

import tensorflow as tf
#!pip install tflearn

import tflearn

from tflearn.layers.core import input_data, dropout, fully_connected

from tflearn.layers.normalization import batch_normalization

from tflearn.layers.conv import conv_2d, upsample_2d, max_pool_2d, avg_pool_2d, conv_3d, grouped_conv_2d, upsample_2d,max_pool_3d, conv_2d_transpose, atrous_conv_2d, upscore_layer, resnext_block

from tflearn.layers.normalization import local_response_normalization

from tflearn.layers.estimator import regression

from tflearn.layers.recurrent import lstm

from tflearn.layers.embedding_ops import embedding

from tflearn.layers.merge_ops import merge



from tflearn.objectives import softmax_categorical_crossentropy





class Model:

    # Building convolutional network



    input_shape = None

    output_shape = None

    encoded_network = None



    def __init__(self,input_shape,output_shape):

        self.input_shape = input_shape

        self.output_shape = output_shape



    def inception(self,network, filter_num,activation='relu', regularizer=None,strides=1,trainable=True):



        network1 = conv_2d(network, int(filter_num/3), 1, activation='relu',strides=1,regularizer=regularizer,trainable=trainable)

        network1 = conv_2d(network1, filter_num, 5, activation='relu',strides=strides,regularizer=regularizer,trainable=trainable)

        network2 = conv_2d(network, int(filter_num/3), 1, activation='relu',strides=1,regularizer=regularizer,trainable=trainable)

        network2 = conv_2d(network2, filter_num, 3, activation='relu',strides=strides,regularizer=regularizer,trainable=trainable)

        network3 = conv_2d(network, int(filter_num/3), 1, activation='relu',strides=1,regularizer=regularizer,trainable=trainable)

        network3 = conv_2d(network3, filter_num, 2, activation='relu',strides=strides,regularizer=regularizer,trainable=trainable)

        # network4 = max_pool_2d(network, kernel_size=2, strides=strides)

        # network5 = max_pool_2d(network, kernel_size=3, strides=strides)

        # network6 = max_pool_2d(network, kernel_size=5, strides=strides)

        network = merge([network1, network2, network3] , mode='concat', axis=3)



        return network



    def load_model(self):



        network = input_data(shape=self.input_shape, name='input')



        network = self.inception(network, 64)

        network = conv_2d(network, 128, 3, activation='relu')

        network = conv_2d(network, 128, 3, activation='relu')

        network = self.inception(network,64,strides=2)



        network = conv_2d(network, 128, 3, activation='relu')

        network = conv_2d(network, 128, 3, activation='relu')

        network = conv_2d(network, 128, 3, activation='relu',strides=2)

        network = self.inception(network, 96)



        network = conv_2d(network, 256, 3, activation='relu')

        network = conv_2d(network, 256, 3, activation='relu')

        network = conv_2d(network, 256, 3, activation='relu',strides=2)

        network = self.inception(network, 96)



        network = conv_2d(network, 256, 3, activation='relu')

        network = conv_2d(network, 256, 3, activation='relu')

        network = conv_2d(network, 256, 3, activation='relu', strides=2)

        network = self.inception(network, 128)



        network = conv_2d(network, 512, 3, activation='relu')

        network = conv_2d(network, 512, 3, activation='relu')

        network = conv_2d(network, 512, 3, activation='relu')

        network = self.inception(network, 128)



        network = fully_connected(network, 512, activation='relu')

        network = dropout(network, 0.8)

        network = fully_connected(network, 6192, activation='relu')

        network = dropout(network, 0.8)

        network = fully_connected(network, 4096, activation='relu')

        network = dropout(network, 0.8)



        network = fully_connected(network, self.output_shape, activation='relu')



        return network



curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [2]:
import cv2

import csv

import gc

import time

from PIL import Image

from tensorflow.python.training.training_util import global_step

from tflearn import Momentum, Adam

from tflearn.optimizers import Optimizer, RMSProp
##################
import cv2

import numpy as np

# import matplotlib.pyplot as plt

import tflearn

import tensorflow as tf

from tensorflow.contrib.losses import log_loss, absolute_difference, mean_pairwise_squared_error

from tflearn.metrics import accuracy_op

from tflearn.objectives import softmax_categorical_crossentropy, categorical_crossentropy, mean_square,binary_crossentropy, roc_auc_score, hinge_loss

import tflearn

In [3]:
def rmse(y_pred, y_true):

    error = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_pred, y_true))))

    return error



def accuracy_error(y_pred, y_true):

    error = tf.sqrt(tf.subtract(y_pred, y_true))

    return error

In [4]:
import os 
import glob
os.chdir('C:\\Users\\Ron\\Desktop\\ML_proj\\Brain_cancer\\data')
files=glob.glob('*.mat')

In [5]:
import h5py
import cv2 
X=[]
Y=[]
data=[]
count=0
#a = 1
for file in files:
    with h5py.File(file,'r') as f:
        variables=f.items()
        for var in variables:
            name= var[0]
            data=var[1]
            if (np.array(data['tumorMask']).shape[1] == 256):
                count+=1
                #print 
                (count)
            else:
                d=data['image']
                X.append(cv2.resize(np.array(d),dsize=(64,64),interpolation=cv2.INTER_AREA))

                #X.append(np.array(d))
                #X.append(np.array(data['tumorMask']))
                Y.append(data['label'][0][0])
                
                #a = data['label'][0][0]
                

In [6]:
# # from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler() #  scaling the data using minmax scaler for better performance
# X= scaler.fit_transform(X)
X=np.array(X).reshape(-1,64,64, 1)
X = X.astype('float32')
X = X / 255.


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
X_train.shape 

(2439, 64, 64, 1)

In [8]:
np.array(Y_train).shape

(2439,)

In [ ]:
frame_height = X_train.shape[0]

frame_width = X_train.shape[1]

input_image = np.reshape(X_train, (-1, frame_height, frame_width, 1))



In [ ]:
output_height = np.array(Y_train).shape[0]

output_width = 1
Y_train=np.array(Y_train)
output_image = np.reshape(Y_train, (-1, output_height * output_width * 1))

In [ ]:
Model = Model(input_shape=[None, frame_height, frame_width, 1], output_shape=output_height * output_width * 3)

network = Model.load_model()

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
momentum = Momentum(learning_rate=0.1, lr_decay=0.96, decay_step=100)

adam = Adam(learning_rate= 0.1)

rms = RMSProp(learning_rate=0.01, decay=0.9)



net = tflearn.regression(network, optimizer=adam, shuffle_batches=True, metric='accuracy', batch_size=32,loss=mean_square)

model = tflearn.DNN(net, tensorboard_verbose=1)



#model.load(X,weights_only=True)

model.fit(input_image, output_image, n_epoch=10, show_metric=True,batch_size = 10, snapshot_step=len(input_image), validation_set=0.2,snapshot_epoch=True)

---------------------------------
Run id: 2UOZRY
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 51
Validation samples: 13
--


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\Ron\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Ron\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ron\Anaconda3\lib\site-packages\tflearn\data_flow.py", line 187, in fill_feed_dict_queue
    data = self.retrieve_data(batch_ids)
  File "C:\Users\Ron\Anaconda3\lib\site-packages\tflearn\data_flow.py", line 222, in retrieve_data
    utils.slice_array(self.feed_dict[key], batch_ids)
  File "C:\Users\Ron\Anaconda3\lib\site-packages\tflearn\utils.py", line 187, in slice_array
    return X[start]
IndexError: index 5 is out of bounds for axis 0 with size 1

